## Import all required libraries and functions 

In [1]:
# Import necessary libraries
import os
import time
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Lambda, Flatten, GlobalMaxPooling2D
from tensorflow.keras.applications import VGG16, ResNet50, InceptionV3

import torch
from torchvision.models.detection import fasterrcnn_resnet50_fpn, retinanet
from torchvision.transforms import functional as F


<b> saving the Start Time

In [2]:
main_start_time=time.time()

## Code for CNN
<b> 1. Setup and Preprocessing

In [3]:

# Define the root folder
root_folder = '/Users/ravkothu/Documents/Personal_items_at_Oracle/Master_Degree/University_of_San_Diego/Online_Masters/MS_in_Applied_AI/Subjects_and_Resources/AAI-501_Introduction_to_AI/AAI-501_Final_Team_Project/pneumonia_detection/dataset/chest_xray_nrm' # 'C:/Users/Soumiz/Downloads/Dummy root'

# Get the subfolders in the root folder
subfolders = [f for f in os.listdir(root_folder) if os.path.isdir(os.path.join(root_folder, f))]

# Assign subfolders to train_folder and test_folder based on their names
for folder in subfolders:
    if 'train' in folder.lower():
        train_folder = os.path.join(root_folder, folder)
        print(f"Found Train Folder --->\t {train_folder}")
    elif 'test' in folder.lower():
        test_folder = os.path.join(root_folder, folder)
        print(f"Found Test Folder --->\t {test_folder}")

# Define class labels dynamically from folder names
class_labels = {}
for element in subfolders:
    element_path = os.path.join(root_folder, element)  # Path identified
    for idx, folder in enumerate(os.listdir(element_path)):
        key=f"{element}_{folder}"
        if 'normal' in folder.lower():
            class_labels[key] = 0
        else:
            class_labels[key] = 1
            
print("\n\t", class_labels)


Found Test Folder --->	 /Users/ravkothu/Documents/Personal_items_at_Oracle/Master_Degree/University_of_San_Diego/Online_Masters/MS_in_Applied_AI/Subjects_and_Resources/AAI-501_Introduction_to_AI/AAI-501_Final_Team_Project/pneumonia_detection/dataset/chest_xray_nrm/test_nrm
Found Train Folder --->	 /Users/ravkothu/Documents/Personal_items_at_Oracle/Master_Degree/University_of_San_Diego/Online_Masters/MS_in_Applied_AI/Subjects_and_Resources/AAI-501_Introduction_to_AI/AAI-501_Final_Team_Project/pneumonia_detection/dataset/chest_xray_nrm/train_nrm

	 {'test_nrm_NORMAL_nrm': 0, 'test_nrm_PNEUMONIA_nrm': 1, 'train_nrm_NORMAL_nrm': 0, 'train_nrm_PNEUMONIA_nrm': 1}


## CNN with Adaptive Pooling

In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Lambda, Flatten
from tensorflow.keras.models import Model

# CNN with Global Average Pooling (similar to adaptive pooling)
def build_cnn_with_adaptive_pooling(input_shape):
    inputs = Input(shape=input_shape)
    x = Conv2D(32, (3, 3), activation='relu')(inputs)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = GlobalAveragePooling2D()(x)  # Global Average Pooling instead of Adaptive Pooling
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    outputs = Dense(2, activation='softmax')(x)  # Assuming binary classification
    model = Model(inputs, outputs)
    return model

# Example usage
if __name__ == "__main__":
    input_shape = (224, 224, 3)  # Example input shape
    cnn_model = build_cnn_with_adaptive_pooling(input_shape)
    cnn_model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 110,018 (429.76 KB)

 Trainable params: 110,018 (429.76 KB)

 Non-trainable params: 0 (0.00 B)

### Load and Preprocess Images

In [5]:
# # root_folder = 'C:/Users/Soumiz/Downloads/AII 501 Project Pneumonia/ChestXRay2017_nrm'

# Define ImageDataGenerator for training and testing
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Create the data generators
train_generator = train_datagen.flow_from_directory(
    os.path.join(root_folder, 'train_nrm'),
    target_size=(224, 224),
    batch_size=8,                                   # Batch size can be changed to bigger 
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    os.path.join(root_folder, 'test_nrm'),
    target_size=(224, 224),
    batch_size=8,                                   # Batch size can be changed to bigger 
    class_mode='categorical')


Found 5232 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


### Get Pre-trained Model

In [6]:
input_shape = (224, 224, 3)
model_name = 'CNN'   
model = cnn_model


### Compile and Train the Model

In [8]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

start_time=time.time()
# Train the model
print("start Time=",time.time()-start_time)

model.fit(train_generator, epochs=50, validation_data=test_generator)   # change epoch to higher value for proper evaluation

print("Time taken to Compile and Train=",time.time()-start_time)

start Time= 8.106231689453125e-05
Epoch 1/50
654/654 ━━━━━━━━━━━━━━━━━━━━ 81s 123ms/step - accuracy: 0.7429 - loss: 0.5561 - val_accuracy: 0.6619 - val_loss: 0.5695
Epoch 2/50
654/654 ━━━━━━━━━━━━━━━━━━━━ 81s 123ms/step - accuracy: 0.7981 - loss: 0.4469 - val_accuracy: 0.6763 - val_loss: 0.5847
Epoch 3/50
654/654 ━━━━━━━━━━━━━━━━━━━━ 81s 123ms/step - accuracy: 0.8463 - loss: 0.3528 - val_accuracy: 0.6875 - val_loss: 0.6991
Epoch 4/50
654/654 ━━━━━━━━━━━━━━━━━━━━ 80s 122ms/step - accuracy: 0.8677 - loss: 0.3010 - val_accuracy: 0.7147 - val_loss: 0.6570
Epoch 5/50
654/654 ━━━━━━━━━━━━━━━━━━━━ 81s 124ms/step - accuracy: 0.8734 - loss: 0.2863 - val_accuracy: 0.7628 - val_loss: 0.5351
Epoch 6/50
654/654 ━━━━━━━━━━━━━━━━━━━━ 81s 123ms/step - accuracy: 0.8810 - loss: 0.2731 - val_accuracy: 0.7756 - val_loss: 0.4934
Epoch 7/50
654/654 ━━━━━━━━━━━━━━━━━━━━ 83s 126ms/step - accuracy: 0.8920 - loss: 0.2512 - val_accuracy: 0.7260 - val_loss: 0.6560
Epoch 8/50
654/654 ━━━━━━━━━━━━━━━━━━━━ 79s 120ms

654/654 ━━━━━━━━━━━━━━━━━━━━ 80s 122ms/step - accuracy: 0.9327 - loss: 0.1613 - val_accuracy: 0.7548 - val_loss: 0.6603
Time taken to Compile and Train= 4142.029619932175


### Evaluate the Model

In [9]:
# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")

78/78 ━━━━━━━━━━━━━━━━━━━━ 4s 57ms/step - accuracy: 0.7512 - loss: 0.6461
Test accuracy: 0.754807710647583


### Evaluation time

In [11]:
Total_time=time.time()-main_start_time
print("Total Execution time=",Total_time)

Total Execution time= 4251.450577020645


## Pre-trained Models [VGG16] with Global Pooling

import tensorflow as tf
from tensorflow.keras.applications import VGG16, ResNet50, InceptionV3
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Pre-trained Models with Global Pooling
def get_pretrained_model(model_name, input_shape):
    if model_name == 'vgg16':
        base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
        x = base_model.output
        x = GlobalAveragePooling2D()(x)  # Global Average Pooling
    elif model_name == 'resnet50':
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
        x = base_model.output
        x = GlobalAveragePooling2D()(x)  # Global Average Pooling
    elif model_name == 'inceptionv3':
        base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)
        x = base_model.output
        x = GlobalAveragePooling2D()(x)  # Global Average Pooling
    else:
        raise ValueError('Unsupported model name.')
    x = Dense(128, activation='relu')(x)
    outputs = Dense(2, activation='softmax')(x)  # Assuming binary classification
    model = Model(base_model.input, outputs)
    return model

# Example usage
if __name__ == "__main__":
    input_shape = (224, 224, 3)  # Example input shape
    model_name = 'vgg16'  # Change to 'resnet50' or 'inceptionv3' as needed
    pretrained_model = get_pretrained_model(model_name, input_shape)
    pretrained_model.summary()


## Object Detection Models

import torch
from torchvision.models.detection import fasterrcnn_resnet50_fpn, retinanet_resnet50_fpn

# Object Detection Models
def get_object_detection_model(model_name):
    if model_name == 'fasterrcnn':
        model = fasterrcnn_resnet50_fpn(weights=True)
    elif model_name == 'retinanet':
        model = retinanet_resnet50_fpn(weights=True)
    else:
        raise ValueError('Unsupported object detection model.')
    return model

# Example usage
if __name__ == "__main__":
    detection_model_name = 'fasterrcnn'  # Change to 'retinanet' as needed
    detection_model = get_object_detection_model(detection_model_name)
    print(f"Object detection model ({detection_model_name}) loaded.")
